# Basic Use of FOLIO with Jupyter Notebooks
To illustrate the use of [Jupyter notebooks](https://jupyter.org/) with [FOLIO](https://www.folio.org/), this notebook uses Ebsco's [FolioClient](https://github.com/FOLIO-FSE/FolioClient) and the [requests](https://requests.readthedocs.io/en/latest/) HTTP Python package.

It is helpful have the Okapi API documentation readily available at https://dev.folio.org/reference/api/ when using the FolioClient or direct Okapi interactions.

## Installing Locally

1. Download and Install [Python 3.10](https://www.python.org/) or later for your platform
1. Create a virtual environment: `python3 -m venv folio-env`
1. Activate your environment (for mac or Linux) `source folio-env/bin/activate`
1. Install the following dependencies:
    - Jupyter Lab `pip install jupyterlab`
    - Requests `pip install requests`
    - FolioClient `pip install folioclient`
1. Start Jupyter lab `~/% jupyter-lab` from the root directory

## Create a FolioClient Instance

In [ ]:
import os
okapi_url = os.getenv("OKAPI_URL")
user = os.getenv("OKAPI_USER")
password = os.getenv("OKAPI_PASSWORD")
tenant = os.getenv("TENANT")

In [ ]:
from folioclient import FolioClient

FolioClient?

In [ ]:
folio_client = FolioClient(
    okapi_url,
    tenant,
    user,
    password)

## Helpful Methods in FolioClient
The Folio Client has a number of convenience methods that wrap specific Okapi endpoints for reference data used in different modules. 

### Locations

In [ ]:
for row in folio_client.locations:
    print(row['code'], row['id'])

### Identifier Types

In [ ]:
for row in folio_client.identifier_types:
    print(row['name'], row['id'])

## Retrieve specific Resources
With the [Inventory Okapi APIs](https://s3.amazonaws.com/foliodocs/api/mod-inventory/p/inventory.html#inventory_instances__instanceid__get), we can use the folio_client's `folio_get` method to retrieve Instance, Holdings, and Items records from FOLIO.

In [ ]:
folio_client.folio_get?

In [ ]:
instance = folio_client.folio_get("/inventory/instances/56270f55-b522-5477-8308-2024345f1e10")

In [ ]:
instance

With the instance's identifiers provide values and the identifier types as a UUID, we don't have from the record a human recognizable labels for these identifier types. We can easily use `folio_client.identifier_types` method and print them in a Jupyter notebook cell.

In [ ]:
# Create an identifier type dictionary to map the uuid to the label
identifier_lookup = dict()
for row in folio_client.identifier_types:
    identifier_lookup[row['id']] = row['name']

In [ ]:
for id_row in instance['identifiers']:
    print(id_row['value'], identifier_lookup[id_row['identifierTypeId']])

In [ ]:
holdings_record = folio_client.folio_get("/holdings-storage/holdings/f7365adc-7a27-50cb-b575-88b858d01057")

In [ ]:
holdings_record

In [ ]:
instance['id'] == holdings_record['instanceId']

In [ ]:
item = folio_client.folio_get("/inventory/items/29b59836-0359-50e6-891a-587cb7b84928")

In [ ]:
item

In [ ]:
item['holdingsRecordId'] == holdings_record['id']

## Alternative Okapi Interactions with requests
The [requests](https://requests.readthedocs.io/en/latest/) Python package is used by the FolioClient for communicating with Okapi, however, you can directly use `requests` to query Okapi.

In [ ]:
import requests

course_listings_result = requests.get(f"{folio_client.okapi_url}/coursereserves/courselistings",
                                      headers=folio_client.okapi_headers)

In [ ]:
course_listings_result

In [ ]:
course_listings_result.json()['courseListings']